***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [27]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-d0cf  GCE       2                                       RUNNING  us-central1-c


# Imports & Setup

In [28]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [29]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  9 16:20 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [31]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [32]:
spark

In [34]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket_itamar'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name.endswith("preprocessed.parquet"):
        paths.append(full_path+b.name)
print(paths)

['gs://bucket_itamar/multistream10_preprocessed.parquet', 'gs://bucket_itamar/multistream11_part2_preprocessed.parquet', 'gs://bucket_itamar/multistream11_preprocessed.parquet', 'gs://bucket_itamar/multistream12_part2_preprocessed.parquet', 'gs://bucket_itamar/multistream12_preprocessed.parquet', 'gs://bucket_itamar/multistream13_part2_preprocessed.parquet', 'gs://bucket_itamar/multistream13_preprocessed.parquet', 'gs://bucket_itamar/multistream14_part2_preprocessed.parquet', 'gs://bucket_itamar/multistream14_preprocessed.parquet', 'gs://bucket_itamar/multistream15_part2_preprocessed.parquet', 'gs://bucket_itamar/multistream15_part3_preprocessed.parquet', 'gs://bucket_itamar/multistream15_preprocessed.parquet', 'gs://bucket_itamar/multistream16_part2_preprocessed.parquet', 'gs://bucket_itamar/multistream16_part3_preprocessed.parquet', 'gs://bucket_itamar/multistream16_preprocessed.parquet', 'gs://bucket_itamar/multistream17_part2_preprocessed.parquet', 'gs://bucket_itamar/multistream17

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [35]:
parquetFile = spark.read.parquet(*paths)

#rdd for each type of index that we need (title , text(body), anchor_text)
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_body_pairs = parquetFile.select("text", "id").rdd
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd


# need to do little manipulation on the anchor text rdd
def make_string_from_anchor(input_string :str) -> str:
    pattern = r"(?<=text=\')[^']+(?=\')"
    matches = re.findall(pattern, input_string)
    string_back = ""
    for term in matches:
        string_back += term + " "

    return string_back
doc_anchor_pairs = doc_anchor_pairs.map(lambda x: (Row(anchor_text=make_string_from_anchor(str(x[0]).lower()),id=x[1])))

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [36]:
# Count number of wiki pages
n = parquetFile.count()
print(n)

6348910


Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [37]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [38]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [39]:
from inverted_index_gcp import InvertedIndex

# Func for the implementation of the invertIndex

In [40]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  list_return= []
  dictio= Counter()
  for i in tokens:
    if i in all_stopwords:
      continue
    elif i in dictio:
      dictio.update({i:1})
    else:
      dictio[i]=1
  for j in dictio:
    list_return.append((j,(id,dictio[j])))
  return list_return


def reduce_word_counts(unsorted_pl):

  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
# YOUR CODE HERE
  unsorted_pl=sorted(unsorted_pl, key=lambda x: x[0])
  return unsorted_pl


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.map(lambda rdd1: (rdd1[0],len(rdd1[1])))


def partition_postings_and_write(postings, bucket_to_save):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  rdd_expbefore_reduce= postings.map(lambda tup:(token2bucket_id(tup[0]),[(tup[0],tup[1])]))
  rdd_reduce= rdd_expbefore_reduce.reduceByKey(lambda a, b: a+b)
  return rdd_reduce.map(lambda tup:InvertedIndex.write_a_posting_list(tup, bucket_to_save))

In [41]:
# save in dict for each doc_id the title
title_dict = {}
for title, doc_id in doc_title_pairs.collect():
    title_dict[doc_id] = title

In [42]:
word = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
def get_doc_len(text_raw, doc_id):
    return doc_id, len(list(word.finditer(text_raw.lower())))

pair = doc_body_pairs.map(lambda x : get_doc_len(x[0],x[1]))
doc_len = Counter()
for doc_id,len_d in pair.collect():
    doc_len[doc_id] = len_d
print(doc_len[60283633])

2806


# Indexing Magic

In [44]:
def make_inverted_index(part_name: str) ->None:
    """
     func that create for each type : body , title .... inverter index
    Args:
      part_name: String body ,title ,anchor

    Returns: None

    """
    word_counts = None
    bucket_name_to_save = ""
    if part_name == "body":
      # word counts map
      bucket_name_to_save = "bucket_itamar_body"
      word_counts = doc_body_pairs.flatMap(lambda x: word_count(x[0], x[1]))

    elif part_name == "title":
      # word counts map
      bucket_name_to_save = "bucket_itamar_title"
      word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))

    elif part_name == "anchor":
      # word counts map
      bucket_name_to_save = "bucket_itamar_anchor"
      word_counts = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))

    postings_filtered = word_counts.groupByKey().mapValues(reduce_word_counts)
    # filtering postings and calculate df
    if part_name == "body":
        # word counts map
        postings_filtered = postings_filtered.filter(lambda x: len(x[1])>50)

    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()
    # partition posting lists and write out
    _ = partition_postings_and_write(postings_filtered, bucket_name_to_save).collect()
    # collect all posting lists locations into one super-set
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name_to_save, prefix='postings_gcp'):
      if not blob.name.endswith("pickle"):
        continue
      with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
          super_posting_locs[k].extend(v)

      # Create inverted index instance
    inverted = InvertedIndex()
    # adding the title of all the docs
    inverted.title = title_dict
    if part_name == "body":
        inverted.doc_len = doc_len
    # save the len of the entire corpus
    inverted.n = n
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df_dict
    # write the global stats out
    inverted.write_index('.', f'index_{part_name}')
    # upload to gs
    index_src = f"index_{part_name}.pkl"
    index_dst = f'gs://{bucket_name_to_save}/postings_gcp/{index_src}'
    !gsutil cp $index_src $index_dst

In [46]:
# make all inverters index
make_inverted_index("title")
make_inverted_index("anchor")
make_inverted_index("body")


23/01/09 17:15:08 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 2.0 in stage 16.0 (TID 1253) (cluster-d0cf-w-1.c.manifest-glyph-370720.internal executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_13181/1805557408.py", line 100, in <lambda>
  File "/hadoop/yarn/nm-local-dir/usercache/root/appcache/application_1673281177178_0002/container_1673281177178_0002_01_000003/inverted_index_gcp.py", line 190, in write_a_posting_list
    with

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 34 in stage 16.0 failed 4 times, most recent failure: Lost task 34.3 in stage 16.0 (TID 1301) (cluster-d0cf-w-1.c.manifest-glyph-370720.internal executor 7): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_13181/1805557408.py", line 100, in <lambda>
  File "/hadoop/yarn/nm-loc

23/01/09 17:15:13 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 44.1 in stage 16.0 (TID 1308) (cluster-d0cf-w-1.c.manifest-glyph-370720.internal executor 7): TaskKilled (Stage cancelled)
23/01/09 17:15:13 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 45.1 in stage 16.0 (TID 1313) (cluster-d0cf-w-1.c.manifest-glyph-370720.internal executor 3): TaskKilled (Stage cancelled)
23/01/09 17:15:13 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 20.3 in stage 16.0 (TID 1306) (cluster-d0cf-w-0.c.manifest-glyph-370720.internal executor 9): TaskKilled (Stage cancelled)
23/01/09 17:15:13 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 3.3 in stage 16.0 (TID 1311) (cluster-d0cf-w-0.c.manifest-glyph-370720.internal executor 9): TaskKilled (Stage cancelled)
23/01/09 17:15:13 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 7.3 in stage 16.0 (TID 1305) (cluster-d0cf-w-0.c.manifest-glyph-370720.internal executor 9): TaskKilled (Stage cancelled)
23/01/0

In [43]:
print(doc_len.most_common(10))

[(21013155, 54394), (32770618, 50348), (2045778, 48749), (49770190, 44848), (1357325, 44119), (299533, 41571), (63635969, 41131), (47807310, 40974), (40566501, 40733), (35797415, 39896)]


# PageRank

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  edges= pages.flatMap(lambda k:map(lambda p:(k['id'],p[0]),k['anchor_text'])).distinct()
  vertices_1= pages.map(lambda k:k[0])
  vertices_2_anchor=pages.flatMap(lambda k:k[1])
  vertices_from_anchor= vertices_2_anchor.map(lambda k:k[0])
  vertices_union= vertices_1.union(vertices_from_anchor)
  vertices= vertices_union.distinct().map(lambda k:(k,))

  return edges, vertices

In [ ]:
#pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
#
# pages_links = spark.read.parquet(*paths).select("id", "anchor_text").rdd
# # construct the graph
# edges, vertices = generate_graph(pages_links)
# # compute PageRank
# edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
# verticesDF = vertices.toDF(['id']).repartition(124, 'id')
# g = GraphFrame(verticesDF, edgesDF)
# pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
# pr = pr_results.vertices.select("id", "pagerank")
# # pr = pr.sort(col('pagerank').desc())
# # save the page rank dict
# pr = pr.toPandas().to_dict()
# pr = pd.DataFrame(pr)
# dict_pr = dict(zip(pr.id, pr.pagerank))
# #save the dict in pkl
# with open('page_rank.pkl', 'wb') as file:
#   pickle.dump(dict_pr, file)


# Page view

In [ ]:
# # Paths
# # Using user page views (as opposed to spiders and automated traffic) for the
# # month of August 2021
# pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
# p = Path(pv_path)
# pv_name = p.name
# pv_temp = f'{p.stem}-4dedup.txt'
# pv_clean = f'{p.stem}.pkl'
# # Download the file (2.3GB)
# !wget -N $pv_path
# # Filter for English pages, and keep just two fields: article ID (3) and monthly
# # total number of page views (5). Then, remove lines with article id or page
# # view values that are not a sequence of digits.
# !bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# # Create a Counter (dictionary) that sums up the pages views for the same
# # article, resulting in a mapping from article id to total page views.
# wid2pv = Counter()
# with open(pv_temp, 'rt') as f:
#   for line in f:
#     parts = line.split(' ')
#     wid2pv.update({int(parts[0]): int(parts[1])})
# # write out the counter as binary file (pickle it)
# with open(pv_clean, 'wb') as f:
#   pickle.dump(wid2pv, f)


In [ ]:
# def read_pkl_file_form_bucket(file_name, name_bucket):
#     """
#         func that read pkl file from the bucket
#     Args:
#         name_bucket: name of the bucket
#         file_name: the name of the pkl file

#     Returns:
#             dict
#     """
#     # access to the bucket
#     client = storage.Client()
#     bucket = client.get_bucket(name_bucket)
#     blob = bucket.get_blob(f'postings_gcp/{file_name}.pkl')
#     with blob.open("rb") as pkl_file:
#         return pickle.load(pkl_file)
# df = read_pkl_file_form_bucket('index_body', 'bucket_itamar_body').df
# for key in df:
#     print(key)
#     print(df[key])